In [2]:
# Import the related libraries
import sklearn
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import collections
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, StratifiedShuffleSplit, cross_validate, RepeatedStratifiedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import LeaveOneOut
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn import preprocessing

In [3]:
# import the dataset
train = pd.read_csv('data/feature_matrix_prcc_cyst_patient.csv')
train.shape

(102, 39)

In [4]:
train.head()

,label,patient,HIST_feature_1,HIST_feature_2,HIST_feature_3,HIST_feature_4,HIST_feature_5,HIST_feature_6,GLRL_feature_1,GLRL_feature_2,...,GLCM_feature_11,GLCM_feature_12,GLCM_feature_13,GLCM_feature_14,GLCM_feature_15,GLCM_feature_16,GLCM_feature_17,GLCM_feature_18,GLCM_feature_19,GLCM_feature_20
0,0,-1,1.004463,0.017497,30.762821,978.899962,0.997622,0.020881,0.000060,19888.566406,...,24.762013,2.328071,97.768728,-0.691609,4.978020,-0.691625,0.000792,0.051091,0.999835,0.999939
1,0,-2,1.005222,0.019843,27.500449,771.798114,0.997195,0.023553,0.000056,19060.814766,...,27.877981,2.383049,110.918460,-0.691346,4.366590,-0.691380,0.000942,0.055692,0.999845,0.999946
2,0,-3,1.007252,0.033582,25.525061,659.494716,0.996838,0.025818,0.000037,27597.260906,...,44.825221,2.522579,176.832785,-0.691123,6.432728,-0.691177,0.001067,0.059233,0.999845,0.999929
3,0,-4,1.010101,0.037297,20.231333,431.438776,0.994152,0.048867,0.000080,17024.387670,...,50.950466,2.736830,198.358401,-0.689474,9.692951,-0.689497,0.002018,0.081287,0.999612,0.999876
4,0,-5,1.010250,0.039945,19.655716,391.075428,0.994702,0.041337,0.000052,19882.968501,...,53.943578,2.742546,213.816780,-0.689803,6.217160,-0.689894,0.001820,0.077223,0.999781,0.999925


In [5]:
y_train = train['label'].ravel()                     # get the labels
# xtrain = train.drop(['label'], axis = 1)
patient = train['patient'].ravel()                   # get the patient numbers   
xtrain = train.drop(['patient', 'label'], axis = 1)  # get the training set
le = preprocessing.LabelEncoder()                    # transform the label into the dummies
y_train = le.fit_transform(y_train)

In [6]:
selection = SelectPercentile(f_classif, percentile=100).fit(xtrain, y_train) # sort the features by p-values
xtrain_c = selection.transform(xtrain)
selection.pvalues_


array([5.74665761e-02, 2.61443616e-02, 1.49682692e-02, 1.09595100e-02,
       1.07031033e-01, 8.31169965e-02, 2.21386868e-03, 2.43833492e-06,
       9.64976944e-02, 2.19141427e-01, 1.04521442e-01, 9.03732109e-02,
       1.22208698e-01, 3.18142758e-03, 6.55269280e-04, 1.80665980e-06,
       1.44451855e-05, 2.88429835e-02, 5.84882662e-04, 6.38000126e-02,
       5.10113588e-03, 1.19854356e-02, 1.30994430e-02, 1.06033465e-01,
       1.06386097e-01, 9.00808411e-02, 1.06998270e-01, 2.71817611e-02,
       5.81276390e-02, 2.87164133e-02, 1.06351108e-01, 5.84882662e-04,
       1.02138166e-01, 1.11511728e-01, 4.73137858e-02, 1.99846176e-02,
       1.50301276e-03])

In [7]:
# get the auc score of each feature
from sklearn import metrics
import numpy as np
import pandas as pd
score = np.zeros([xtrain.shape[1], 1])
for i in range(xtrain.shape[1]):
    fpr, tpr, thresholds = metrics.roc_curve(y_train, xtrain.iloc[:, i], pos_label=1)
    score[i,:] = metrics.auc(fpr, tpr)
list(pd.DataFrame(score)[0])

[0.36099345398536775,
 0.3307662687716596,
 0.5987678090103966,
 0.5991528686946477,
 0.5999229880631498,
 0.3943011166730843,
 0.7015787447054292,
 0.24528301886792453,
 0.39507123604158645,
 0.4485945321524837,
 0.398536773199846,
 0.39776665383134385,
 0.6003080477474008,
 0.6961879091259144,
 0.7112052368117058,
 0.24297266076241816,
 0.2576049287639584,
 0.3365421640354255,
 0.2722371967654986,
 0.42857142857142855,
 0.28147862918752403,
 0.3084328070850982,
 0.33692722371967654,
 0.601078167115903,
 0.398921832884097,
 0.6060839430111667,
 0.601078167115903,
 0.33307662687716594,
 0.36388140161725063,
 0.3350019252984212,
 0.398921832884097,
 0.2722371967654986,
 0.3977666538313438,
 0.4008471313053523,
 0.4008471313053523,
 0.6526761648055448,
 0.7061994609164421]

In [27]:
# get the histogram features
xtrain_hist = xtrain.iloc[:, :6]
xtrain_texture = xtrain.iloc[:, 6:]

In [8]:
# calculate the class weight
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', np.unique(y_train), y_train)
weights, np.histogram(y_train)

(array([0.7244898 , 1.61363636]),
 (array([49,  0,  0,  0,  0,  0,  0,  0,  0, 22], dtype=int64),
  array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])))

In [21]:

l = [5, 10, 20, 40, 60, 80, 100] # the percent of features we use in training
num = 5                          # number of  loops we train
for j in l:
    print('------', j, '-------')
    for i in range(num):
        selection = SelectPercentile(f_classif, percentile=j).fit(xtrain, y_train) # extract certain percentile of features
        xtrain_c = selection.transform(xtrain)
        scaler = StandardScaler().fit(xtrain_c)                                    # normalize the features
        x_train = scaler.transform(xtrain_c)
        scoring ={ "Accuracy":'accuracy',"AUC":'roc_auc'}
        s = StratifiedShuffleSplit(n_splits=3, random_state=i+1)                   # split the dataset into 3 folds
        
        # support vector machine
        params = {'class_weight': ['balanced', None, {0: weights[0], 1: weights[1]}], 'gamma': ['auto', 1e-3, 1e-5, 'scale'],'C': [0.001, 0.01, 0.1, 10, 1, 100, 1000, 10000, 12000]}
        scoring ={ "Accuracy":'accuracy',"AUC":'roc_auc'}
        svc_search = GridSearchCV(SVC(random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        svc_search.fit(x_train, y_train)
        cvres = svc_search.cv_results_
        print('SVM:', svc_search.best_score_, svc_search.best_params_)
        
        # LGBM
        params = {'boosting_type': ['gbdt']}
        lgb_search = GridSearchCV(lgb.LGBMClassifier(class_weight={0: weights[0], 1: weights[1]}, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search.best_score_, lgb_search.best_params_)

        params = {'learning_rate': [0.007,  0.01, 0.015, 0.025, 0.05, 0.1],
                 'n_estimators': range(400, 501, 10)}
        lgb_search0 = GridSearchCV(lgb_search.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search0.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search0.best_score_, lgb_search0.best_params_)

        param_grid = {'max_depth': [-1, 4,5, 6, 7, 8, 9, 10],
                      'num_leaves': range(40, 51, 2)
                 }
        lgb_search1 = GridSearchCV(lgb_search0.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search1.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search1.best_score_, lgb_search1.best_params_)

        param_grid = {'min_child_samples': range(1, 41, 2),
                      'min_child_weight': [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.1],
                      'min_split_gain': [0, 0.01, 0.001, 0.1, 0.2, 0.3, 0.4]
                 }

        lgb_search2 = GridSearchCV(lgb_search1.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search2.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search2.best_score_, lgb_search2.best_params_)

        param_grid = {'subsample': [0.1, 0.2, 0.3, 0.4, 0.5],
                      'colsample_bytree': [0.1, 0.2, 0.3, 0.4, 0.5]
                 }
        lgb_search3 = GridSearchCV(lgb_search2.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search3.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search3.best_score_, lgb_search3.best_params_)

        param_grid = { 'feature_fraction': [0.1, 0.2, 0.3, 0.4, 0.5],
                      'bagging_fraction': [0.1, 0.2, 0.3, 0.4, 0.5]
        }
        lgb_search4 = GridSearchCV(lgb_search3.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search4.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search4.best_score_, lgb_search4.best_params_)

        param_grid = {'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5], 
                      'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]
        }
        lgb_search5 = GridSearchCV(lgb_search4.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search5.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search5.best_score_, lgb_search5.best_params_)

        # random forest
        param_grid = {'n_estimators':range(10, 201, 10),
                  'criterion':['gini','entropy']}
        rf_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search.fit(x_train, y_train)
        print('rf', rf_search.best_score_, rf_search.best_params_)

        param_grid = {'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9],
                      'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9],
                     'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5]}
        rf_search1 = GridSearchCV(rf_search.best_estimator_, param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search1.fit(x_train, y_train)
        print('rf', rf_search1.best_score_, rf_search1.best_params_)

        param_grid = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                      'min_samples_split': range(2, 11, 1),
                      'min_samples_leaf': range(1, 11, 1)}
        rf_search2 = GridSearchCV(rf_search1.best_estimator_, param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search2.fit(x_train, y_train)
        print('rf', rf_search2.best_score_, rf_search2.best_params_)

        # xgboost
        params = {'learning_rate': [0.0001, 0.001, 0.01],
                  'n_estimators': [100, 300, 500, 700],
                  'max_depth' : [2,4,6],
                  'gamma': [0, 0.001, 0.01, 0.1],
                  'subsample': [0.4, 0.7, 1],
                  'colsample_bytree': [0.5, 0.7, 0.9],
                  'scale_pos_weight': [0.1, 0.3, 1, 3]
                 }
        xg_search3 = GridSearchCV(xgb.XGBClassifier(booster='dart', class_weight={0: weights[0], 1: weights[1]}, random_state=0), 
                                  params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        xg_search3.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('xg:', xg_search3.best_score_, xg_search3.best_params_)

        # gradient boosting decision tree
        params = {'n_estimators':range(10, 201, 10), 
          'learning_rate': [0.001, 0.01, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4]
        }
        gb_search1 = GridSearchCV(GradientBoostingClassifier(n_iter_no_change=50, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search1.fit(x_train, y_train)
        cvres = gb_search1.cv_results_
        print('gb', gb_search1.best_score_, gb_search1.best_params_)

        params = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                  'max_leaf_nodes': [None]+list(range(2, 41, 2))}
        gb_search2 = GridSearchCV(gb_search1.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search2.fit(x_train, y_train)
        cvres = gb_search2.cv_results_
        print('gb', gb_search2.best_score_, gb_search2.best_params_)

        params = {'min_samples_split': range(2, 11, 1),
                  'min_samples_leaf': range(1, 11, 1), 
                 'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9]}
        gb_search3 = GridSearchCV(gb_search2.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search3.fit(x_train, y_train)
        cvres = gb_search3.cv_results_
        print('gb', gb_search3.best_score_, gb_search3.best_params_)

        params = {'subsample': [0.6, 0.7, 0.8, 0.9, 1],
                  'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9],
                  'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5]
                }
        gb_search4 = GridSearchCV(gb_search3.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search4.fit(x_train, y_train)
        cvres = gb_search4.cv_results_
        print('gb', gb_search4.best_score_, gb_search4.best_params_)

        # Extra trees
        params = {'n_estimators':range(10, 301, 10)}
        et_search1 = GridSearchCV(ExtraTreesClassifier(class_weight={0: weights[0], 1: weights[1]}, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search1.fit(x_train, y_train)
        cvres = et_search1.cv_results_
        print('et', et_search1.best_score_, et_search1.best_params_)

        params = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                  'min_samples_split': range(2, 11, 1),
                  'min_samples_leaf': range(1, 11, 1)
                }
        et_search2 = GridSearchCV(et_search1.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search2.fit(x_train, y_train)
        cvres = et_search2.cv_results_
        print('et', et_search2.best_score_, et_search2.best_params_)

        params = {'max_leaf_nodes': [None]+list(range(2, 41, 2)),
                  'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9]}
        et_search3 = GridSearchCV(et_search2.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search3.fit(x_train, y_train)
        cvres = et_search3.cv_results_
        print('et', et_search3.best_score_, et_search3.best_params_)

        params = {'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5],
                  'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9]}
        et_search4 = GridSearchCV(et_search3.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search4.fit(x_train, y_train)
        cvres = et_search4.cv_results_
        print('et', et_search4.best_score_, et_search4.best_params_)

        # histogram-based gradient boosting
        params = {'learning_rate': [0.001, 0.003, 0.01, 0.03],
                 'max_iter': [40,60,80,100],
                 'max_depth': [None, 2,4,6],
                  'min_samples_leaf': [1, 2, 3],
                  'max_leaf_nodes': range(2, 41, 5)
                 }
        hgb_search2 = GridSearchCV(HistGradientBoostingClassifier(n_iter_no_change=50), 
                                   params,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        hgb_search2.fit(x_train, y_train)
        print('hgb:', hgb_search2.best_score_, hgb_search2.best_params_)

------ 5 -------
SVM: 1.0 {'C': 0.001, 'class_weight': 'balanced', 'gamma': 'auto'}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 {'learning_rate': 0.007, 'n_estimators': 400}
lgb: 0.5 {'max_depth': -1, 'num_leaves': 40}
lgb: 0.8333333333333334 {'min_child_samples': 1, 'min_child_weight': 0.7, 'min_split_gain': 0}
lgb: 0.8333333333333334 {'colsample_bytree': 0.1, 'subsample': 0.1}
lgb: 0.8333333333333334 {'bagging_fraction': 0.1, 'feature_fraction': 0.1}
lgb: 0.8333333333333334 {'reg_alpha': 0, 'reg_lambda': 0}
rf 1.0 {'criterion': 'gini', 'n_estimators': 10}
rf 1.0 {'max_features': 'auto', 'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
rf 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 0.8333333333333334 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.0001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 1}
gb 0.3333333333333333 {'learning_rate': 0.01, 'n_estimators': 40}
gb 0.3333333333333333 {'max_depth': 4, 'max_le

rf 0.6666666666666666 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 1.0 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 3, 'subsample': 0.4}
gb 0.6666666666666666 {'learning_rate': 0.2, 'n_estimators': 20}
gb 0.6666666666666666 {'max_depth': 4, 'max_leaf_nodes': None}
gb 0.6666666666666666 {'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_samples_split': 2}
gb 0.6666666666666666 {'max_features': 'auto', 'min_weight_fraction_leaf': 0, 'subsample': 0.7}
et 0.8333333333333334 {'n_estimators': 20}
et 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
et 1.0 {'max_features': 'auto', 'max_leaf_nodes': None}
et 1.0 {'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
hgb: 1.0 {'learning_rate': 0.03, 'max_depth': None, 'max_iter': 40, 'max_leaf_nodes': 17, 'min_samples_leaf': 2}
SVM: 1.0 {'C': 100, 'class_weight': 'balanced', 'gamma': 'auto'}
lgb: 0.5 {'boosting_type': 'g

et 0.8333333333333334 {'max_features': 'auto', 'max_leaf_nodes': None}
et 0.8333333333333334 {'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
hgb: 1.0 {'learning_rate': 0.001, 'max_depth': 4, 'max_iter': 80, 'max_leaf_nodes': 32, 'min_samples_leaf': 2}
SVM: 0.6666666666666666 {'C': 0.001, 'class_weight': 'balanced', 'gamma': 'auto'}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 {'learning_rate': 0.007, 'n_estimators': 400}
lgb: 0.5 {'max_depth': -1, 'num_leaves': 40}
lgb: 0.8333333333333334 {'min_child_samples': 1, 'min_child_weight': 0.7, 'min_split_gain': 0}
lgb: 1.0 {'colsample_bytree': 0.1, 'subsample': 0.1}
lgb: 1.0 {'bagging_fraction': 0.1, 'feature_fraction': 0.1}
lgb: 1.0 {'reg_alpha': 0, 'reg_lambda': 0}
rf 1.0 {'criterion': 'gini', 'n_estimators': 20}
rf 1.0 {'max_features': 'auto', 'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
rf 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 3}
xg: 0.6666666666666666 {'colsample_bytree': 0.5, 'gamma':

rf 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 0.6666666666666666 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.0001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 0.7}
gb 0.6666666666666666 {'learning_rate': 0.001, 'n_estimators': 10}
gb 0.6666666666666666 {'max_depth': 4, 'max_leaf_nodes': None}
gb 0.6666666666666666 {'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_samples_split': 2}
gb 1.0 {'max_features': 0.1, 'min_weight_fraction_leaf': 0, 'subsample': 0.7}
et 1.0 {'n_estimators': 10}
et 1.0 {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2}
et 1.0 {'max_features': 'auto', 'max_leaf_nodes': None}
et 1.0 {'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
hgb: 0.8333333333333334 {'learning_rate': 0.003, 'max_depth': 2, 'max_iter': 40, 'max_leaf_nodes': 27, 'min_samples_leaf': 2}
SVM: 1.0 {'C': 0.001, 'class_weight': 'balanced', 'gamma': 'auto'}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 

SVM: 1.0 {'C': 0.001, 'class_weight': 'balanced', 'gamma': 'auto'}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 {'learning_rate': 0.007, 'n_estimators': 400}
lgb: 0.5 {'max_depth': -1, 'num_leaves': 40}
lgb: 1.0 {'min_child_samples': 1, 'min_child_weight': 0.8, 'min_split_gain': 0}
lgb: 1.0 {'colsample_bytree': 0.1, 'subsample': 0.1}
lgb: 1.0 {'bagging_fraction': 0.1, 'feature_fraction': 0.1}
lgb: 1.0 {'reg_alpha': 0, 'reg_lambda': 0}
rf 1.0 {'criterion': 'gini', 'n_estimators': 10}
rf 1.0 {'max_features': 'auto', 'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
rf 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 1.0 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.0001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 3, 'subsample': 1}
gb 0.8333333333333334 {'learning_rate': 0.1, 'n_estimators': 20}
gb 0.8333333333333334 {'max_depth': 4, 'max_leaf_nodes': 4}
gb 0.8333333333333334 {'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_

gb 0.5 {'learning_rate': 0.05, 'n_estimators': 40}
gb 0.5 {'max_depth': 4, 'max_leaf_nodes': 2}
gb 0.6666666666666666 {'min_impurity_decrease': 0, 'min_samples_leaf': 2, 'min_samples_split': 2}
gb 1.0 {'max_features': 'log2', 'min_weight_fraction_leaf': 0, 'subsample': 1}
et 1.0 {'n_estimators': 10}
et 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
et 1.0 {'max_features': 'auto', 'max_leaf_nodes': None}
et 1.0 {'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
hgb: 1.0 {'learning_rate': 0.001, 'max_depth': None, 'max_iter': 100, 'max_leaf_nodes': 22, 'min_samples_leaf': 3}
SVM: 1.0 {'C': 0.001, 'class_weight': 'balanced', 'gamma': 0.001}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 {'learning_rate': 0.007, 'n_estimators': 400}
lgb: 0.5 {'max_depth': -1, 'num_leaves': 40}
lgb: 0.8333333333333334 {'min_child_samples': 1, 'min_child_weight': 0.5, 'min_split_gain': 0}
lgb: 1.0 {'colsample_bytree': 0.5, 'subsample': 0.1}
lgb: 1.0 {'bagging_fraction': 0.1, 'featu

lgb: 0.6666666666666666 {'colsample_bytree': 0.1, 'subsample': 0.1}
lgb: 0.6666666666666666 {'bagging_fraction': 0.1, 'feature_fraction': 0.1}
lgb: 0.6666666666666666 {'reg_alpha': 0, 'reg_lambda': 0}
rf 0.6666666666666666 {'criterion': 'gini', 'n_estimators': 140}
rf 1.0 {'max_features': 'log2', 'min_impurity_decrease': 0.1, 'min_weight_fraction_leaf': 0.3}
rf 1.0 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 1.0 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.0001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 0.4}
gb 0.5 {'learning_rate': 0.001, 'n_estimators': 10}
gb 0.5 {'max_depth': 4, 'max_leaf_nodes': None}
gb 0.6666666666666666 {'min_impurity_decrease': 0, 'min_samples_leaf': 3, 'min_samples_split': 10}
gb 1.0 {'max_features': 'auto', 'min_weight_fraction_leaf': 0, 'subsample': 0.9}
et 0.6666666666666666 {'n_estimators': 10}
et 0.6666666666666666 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
et 1.0 {'m

In [17]:
# train the histogram features and the texture features
x = [xtrain_hist, xtrain_texture]
for xtrain in x:
    for i in range(5):
        selection = SelectPercentile(f_classif, percentile=100).fit(xtrain, y_train)
        xtrain_c = selection.transform(xtrain)
        scaler = StandardScaler().fit(xtrain_c)
        x_train = scaler.transform(xtrain_c)
        scoring ={ "Accuracy":'accuracy',"AUC":'roc_auc'}
        s = StratifiedShuffleSplit(n_splits=3, random_state=i+1)

        params = {'class_weight': ['balanced', None, {0: weights[0], 1: weights[1]}], 'gamma': ['auto', 1e-3, 1e-5, 'scale'],'C': [0.001, 0.01, 0.1, 10, 1, 100, 1000, 10000, 12000]}
        scoring ={ "Accuracy":'accuracy',"AUC":'roc_auc'}
        svc_search = GridSearchCV(SVC(random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        svc_search.fit(x_train, y_train)
        cvres = svc_search.cv_results_
        print('SVM:', svc_search.best_score_, svc_search.best_params_)

        params = {'boosting_type': ['gbdt']}
        lgb_search = GridSearchCV(lgb.LGBMClassifier(class_weight={0: weights[0], 1: weights[1]}, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search.best_score_, lgb_search.best_params_)

        params = {'learning_rate': [0.007,  0.01, 0.015, 0.025, 0.05, 0.1],
                 'n_estimators': range(400, 501, 10)}
        lgb_search0 = GridSearchCV(lgb_search.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search0.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search0.best_score_, lgb_search0.best_params_)

        param_grid = {'max_depth': [-1, 4,5, 6, 7, 8, 9, 10],
                      'num_leaves': range(40, 51, 2)
                 }
        lgb_search1 = GridSearchCV(lgb_search0.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search1.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search1.best_score_, lgb_search1.best_params_)

        param_grid = {'min_child_samples': range(1, 41, 2),
                      'min_child_weight': [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.1],
                      'min_split_gain': [0, 0.01, 0.001, 0.1, 0.2, 0.3, 0.4]
                 }

        lgb_search2 = GridSearchCV(lgb_search1.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search2.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search2.best_score_, lgb_search2.best_params_)

        param_grid = {'subsample': [0.1, 0.2, 0.3, 0.4, 0.5],
                      'colsample_bytree': [0.1, 0.2, 0.3, 0.4, 0.5]
                 }
        lgb_search3 = GridSearchCV(lgb_search2.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search3.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search3.best_score_, lgb_search3.best_params_)

        param_grid = { 'feature_fraction': [0.1, 0.2, 0.3, 0.4, 0.5],
                      'bagging_fraction': [0.1, 0.2, 0.3, 0.4, 0.5]
        }
        lgb_search4 = GridSearchCV(lgb_search3.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search4.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search4.best_score_, lgb_search4.best_params_)

        param_grid = {'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5], 
                      'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]
        }
        lgb_search5 = GridSearchCV(lgb_search4.best_estimator_, param_grid,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        lgb_search5.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('lgb:', lgb_search5.best_score_, lgb_search5.best_params_)


        param_grid = {'n_estimators':range(10, 201, 10),
                  'criterion':['gini','entropy']}
        rf_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search.fit(x_train, y_train)
        print('rf', rf_search.best_score_, rf_search.best_params_)

        param_grid = {'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9],
                      'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9],
                     'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5]}
        rf_search1 = GridSearchCV(rf_search.best_estimator_, param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search1.fit(x_train, y_train)
        print('rf', rf_search1.best_score_, rf_search1.best_params_)

        param_grid = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                      'min_samples_split': range(2, 11, 1),
                      'min_samples_leaf': range(1, 11, 1)}
        rf_search2 = GridSearchCV(rf_search1.best_estimator_, param_grid, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        rf_search2.fit(x_train, y_train)
        print('rf', rf_search2.best_score_, rf_search2.best_params_)


        params = {'learning_rate': [0.0001, 0.001, 0.01],
                  'n_estimators': [100, 300, 500, 700],
                  'max_depth' : [2,4,6],
                  'gamma': [0, 0.001, 0.01, 0.1],
                  'subsample': [0.4, 0.7, 1],
                  'colsample_bytree': [0.5, 0.7, 0.9],
                  'scale_pos_weight': [0.1, 0.3, 1, 3]
                 }
        xg_search3 = GridSearchCV(xgb.XGBClassifier(booster='dart', class_weight={0: weights[0], 1: weights[1]}, random_state=0), 
                                  params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        xg_search3.fit(x_train, y_train, eval_metric='auc', early_stopping_rounds=50, eval_set=[(x_train, y_train)], verbose=False)
        print('xg:', xg_search3.best_score_, xg_search3.best_params_)


        params = {'n_estimators':range(10, 201, 10), 
          'learning_rate': [0.001, 0.01, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4]
        }
        gb_search1 = GridSearchCV(GradientBoostingClassifier(n_iter_no_change=50, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search1.fit(x_train, y_train)
        cvres = gb_search1.cv_results_
        print('gb', gb_search1.best_score_, gb_search1.best_params_)

        params = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                  'max_leaf_nodes': [None]+list(range(2, 41, 2))}
        gb_search2 = GridSearchCV(gb_search1.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search2.fit(x_train, y_train)
        cvres = gb_search2.cv_results_
        print('gb', gb_search2.best_score_, gb_search2.best_params_)

        params = {'min_samples_split': range(2, 11, 1),
                  'min_samples_leaf': range(1, 11, 1), 
                 'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9]}
        gb_search3 = GridSearchCV(gb_search2.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search3.fit(x_train, y_train)
        cvres = gb_search3.cv_results_
        print('gb', gb_search3.best_score_, gb_search3.best_params_)

        params = {'subsample': [0.6, 0.7, 0.8, 0.9, 1],
                  'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9],
                  'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5]
                }
        gb_search4 = GridSearchCV(gb_search3.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        gb_search4.fit(x_train, y_train)
        cvres = gb_search4.cv_results_
        print('gb', gb_search4.best_score_, gb_search4.best_params_)


        params = {'n_estimators':range(10, 301, 10)}
        et_search1 = GridSearchCV(ExtraTreesClassifier(class_weight={0: weights[0], 1: weights[1]}, random_state=0), params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search1.fit(x_train, y_train)
        cvres = et_search1.cv_results_
        print('et', et_search1.best_score_, et_search1.best_params_)

        params = {'max_depth':[4,5, 6, 7, 8, 9, 10, 11],
                  'min_samples_split': range(2, 11, 1),
                  'min_samples_leaf': range(1, 11, 1)
                }
        et_search2 = GridSearchCV(et_search1.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search2.fit(x_train, y_train)
        cvres = et_search2.cv_results_
        print('et', et_search2.best_score_, et_search2.best_params_)

        params = {'max_leaf_nodes': [None]+list(range(2, 41, 2)),
                  'max_features': ['auto', None, 'log2', 'sqrt', 0.1, 0.3, 0.5, 0.7, 0.9]}
        et_search3 = GridSearchCV(et_search2.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search3.fit(x_train, y_train)
        cvres = et_search3.cv_results_
        print('et', et_search3.best_score_, et_search3.best_params_)

        params = {'min_weight_fraction_leaf': [0, 0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.5],
                  'min_impurity_decrease':[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 0.9]}
        et_search4 = GridSearchCV(et_search3.best_estimator_, params, cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        et_search4.fit(x_train, y_train)
        cvres = et_search4.cv_results_
        print('et', et_search4.best_score_, et_search4.best_params_)


        params = {'learning_rate': [0.001, 0.003, 0.01, 0.03],
                 'max_iter': [40,60,80,100],
                 'max_depth': [None, 2,4,6],
                  'min_samples_leaf': [1, 2, 3],
                  'max_leaf_nodes': range(2, 41, 5)
                 }
        hgb_search2 = GridSearchCV(HistGradientBoostingClassifier(n_iter_no_change=50), 
                                   params,cv=s.split(x_train, y_train), scoring=scoring, refit='AUC')
        hgb_search2.fit(x_train, y_train)
        print('hgb:', hgb_search2.best_score_, hgb_search2.best_params_)

SVM: 0.8333333333333334 {'C': 100, 'class_weight': None, 'gamma': 1e-05}
lgb: 0.5 {'boosting_type': 'gbdt'}
lgb: 0.5 {'learning_rate': 0.007, 'n_estimators': 400}
lgb: 0.5 {'max_depth': -1, 'num_leaves': 40}
lgb: 0.75 {'min_child_samples': 1, 'min_child_weight': 0.4, 'min_split_gain': 0}
lgb: 0.8333333333333334 {'colsample_bytree': 0.1, 'subsample': 0.1}
lgb: 0.8333333333333334 {'bagging_fraction': 0.1, 'feature_fraction': 0.1}
lgb: 0.8333333333333334 {'reg_alpha': 0, 'reg_lambda': 0}
rf 0.75 {'criterion': 'gini', 'n_estimators': 40}
rf 0.75 {'max_features': 'auto', 'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
rf 0.75 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 0.6666666666666666 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 3, 'subsample': 0.7}
gb 0.75 {'learning_rate': 0.4, 'n_estimators': 10}
gb 0.8333333333333334 {'max_depth': 4, 'max_leaf_nodes': None}
gb 0.833333333333333

rf 0.6666666666666666 {'max_features': 'auto', 'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0.1}
rf 0.6666666666666666 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
xg: 0.6666666666666666 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.0001, 'max_depth': 2, 'n_estimators': 100, 'scale_pos_weight': 3, 'subsample': 0.4}
gb 0.75 {'learning_rate': 0.2, 'n_estimators': 10}
gb 0.5833333333333334 {'max_depth': 4, 'max_leaf_nodes': None}
gb 0.75 {'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_samples_split': 3}
gb 0.8333333333333334 {'max_features': 'auto', 'min_weight_fraction_leaf': 0, 'subsample': 0.9}
et 0.6666666666666666 {'n_estimators': 10}
et 0.6666666666666666 {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
et 0.6666666666666666 {'max_features': 'auto', 'max_leaf_nodes': None}
et 0.6666666666666666 {'min_impurity_decrease': 0, 'min_weight_fraction_leaf': 0}
hgb: 0.8333333333333334 {'learning_rate': 0.001, 'max_depth': 6, '